In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata

# OPTIONAL (recommended for geospatial): project to a metric CRS
from pyproj import Transformer

# OPTIONAL (export): write a GeoTIFF with a CRS
import xarray as xr
import rioxarray  # noqa

# -------------------------------
# 1) Load data: lon, lat, value
# -------------------------------
# Example DataFrame; replace with your own data source
df = pd.DataFrame({
    "lon": [98.95, 99.10, 99.25, 99.40, 99.55],
    "lat": [18.65, 18.70, 18.80, 18.75, 18.90],
    "val": [25.1, 28.3, 27.2, 29.5, 26.8]
})

# -------------------------------
# 2) Project coordinates to a projected CRS (meters)
#    Example: UTM zone 47N for Northern Thailand → EPSG:32647
# -------------------------------
to_utm = Transformer.from_crs("EPSG:4326", "EPSG:32647", always_xy=True)
x_m, y_m = to_utm.transform(df["lon"].values, df["lat"].values)

# -------------------------------
# 3) Build interpolation grid in projected coordinates
#    Define desired resolution (e.g., 500 m per pixel)
# -------------------------------
res = 500.0  # meters
xmin, xmax = x_m.min(), x_m.max()
ymin, ymax = y_m.min(), y_m.max()

# Add small padding
pad = 2 * res
xmin, xmax = xmin - pad, xmax + pad
ymin, ymax = ymin - pad, ymax + pad

# Create regular grid
x_coords = np.arange(xmin, xmax + res, res)
y_coords = np.arange(ymin, ymax + res, res)
gx, gy = np.meshgrid(x_coords, y_coords)

# -------------------------------
# 4) Interpolate with SciPy
# -------------------------------
points = np.c_[x_m, y_m]
values = df["val"].values

# Choose method: 'linear' (robust), 'cubic' (smooth, needs denser points), or 'nearest'
grid_linear = griddata(points, values, (gx, gy), method='cubic')

# Optional: fill NaNs (outside convex hull) using nearest as a fallback
grid_nearest = griddata(points, values, (gx, gy), method='nearest')
grid_filled = np.where(np.isnan(grid_linear), grid_nearest, grid_linear)

# -------------------------------
# 5) (Optional) Export to GeoTIFF with CRS using xarray/rioxarray
# -------------------------------
# y/x → note that rasters conventionally index [row, col] as [y, x]
da = xr.DataArray(
    grid_filled,
    coords={"y": y_coords, "x": x_coords},
    dims=("y", "x"),
    name="interpolated"
)

# Write CRS and transform; for evenly spaced grid, rioxarray infers transform from coords
da = da.rio.write_crs("EPSG:32647")
da.rio.to_raster("interpolated_surface_epsg32647.tif")

print("Saved: interpolated_surface_epsg32647.tif")


Saved: interpolated_surface_epsg32647.tif
